### 环境要求

python 3.12 与最新 langchain 存在兼容问题，所以降级安装3.9版本

```cmd
conda create -n .venv python=3.9
conda activate .venv
```

### 安装依赖
```cmd
conda install ipykernel  // jupyter notebook 需要
conda install python-dotenv -c conda-forge 
conda install langchain -c conda-forge 
conda install langchain-community -c conda-forge
conda install openai -c conda-forge
conda install tiktoken -c conda-forge  // 对话字符缓存使用
```

### 通过 LangChain 使用 OpenAI
三个重要的概念：模型、提示与解释器
#### 模型
从 `langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`，除了`OpenAI` 以外，`chat_models`也集成了其它的对话模型。


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

_ = load_dotenv(find_dotenv())


# 获取环境变量
api_key = os.environ["OPENAI_API_KEY_LOCAL"]
base_url = os.environ["OPENAI_BASE_URL"]

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)

#### 提示模板
在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。使用提示模版，可以让我们更为方便地重复使用设计好的提示。
LangChain还提供了提示模版用于一些常用场景。比如自动摘要、问答、连接到SQL数据库、连接到不同的API。通过使用LangChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

In [ ]:
# 编写 prompt
template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""

# 然后，我们调用`ChatPromptTemplatee.from_template()`函数将
# 上面的提示模版字符`template_string`转换为提示模版`prompt_template`

prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""

customer_email = """
有人违法使用我的电话信息，请马上注销，不然我将投诉12315/信息安全局以及曝光，请2小时马上答复我。
"""

# 使用提示模版
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)

customer_response = chat(customer_messages)
print(customer_response.content)


#### 输出解释器
以上`customer_response`输出为 `str` 类型，如果想要方便的提取信息，需要使用`Langchain`中的输出解释器。

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

template_string = """
对于以下文本，提取出以下信息：

语气：语气是否平和

语言风格：语言风格是否正式

使用以下键将输出格式化为 JSON：
语气
语言风格


文本：{text}

{format_instructions}
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

response_schemas  = [
  ResponseSchema(name="语气", description="语气是否平和"),
  ResponseSchema(name="语言风格", description="语言风格是否正式")
]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print("输出格式规定：",format_instructions)



text = """
有人违法使用我的电话信息，请马上注销，不然我将投诉12315/信息安全局以及曝光，请2小时马上答复我。
"""

# 使用提示模版
message = prompt_template.format_messages(text=text, format_instructions=format_instructions)

customer_response = chat(message)
print("结果类型:", type(customer_response.content))
print(customer_response.content)

# output_dict = output_parser.parse(customer_response.content)
# print("解析后的结果类型:", type(output_dict))




### 储存 Memory
使用 LangChain 中的储存(Memory)模块时，它旨在保存、组织和跟踪整个对话的历史，从而为用户和模型之间的交互提供连续的上下文。

LangChain 提供了多种储存类型。其中:
* 缓冲区储存允许保留最近的聊天消息
* 摘要储存则提供了对整个对话的摘要。
* 实体储存则允许在多轮对话中保留有关特定实体的信息。

这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。

主要的 4 种存储模块：
* 对话缓存储存 (ConversationBufferMemory）
* 对话缓存窗口储存 (ConversationBufferWindowMemory）
* 对话令牌缓存储存 (ConversationTokenBufferMemory）
* 对话摘要缓存储存 (ConversationSummaryBufferMemory）

#### 对话缓存存储
属于缓冲区储存器，用于存储对话上下文信息，包括历史消息、当前消息、历史消息的embedding等。

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试增大该参数。
llm = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})

# 新建一个 ConversationChain Class 实例
# verbose参数设置为True时，程序会输出更详细的信息，以提供更多的调试或运行时信息。
# 相反，当将verbose参数设置为False时，程序会以更简洁的方式运行，只输出关键的信息。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

conversation.predict(input="你好, 我叫池")
conversation.predict(input="1+1 等于几")
conversation.predict(input="我叫什么")

memory.load_memory_variables({})

#### 对话窗口存储
对话缓存窗口储存只保留一个窗口大小的对话。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大。

In [36]:
from langchain.memory import ConversationBufferWindowMemory

# k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  

llm = ChatOpenAI(temperature=0.0, api_key=api_key, base_url=base_url)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )

print("第一轮对话：")
print(conversation.predict(input="你好, 我叫池"))

print("第二轮对话：")
print(conversation.predict(input="我叫什么名字？"))


第一轮对话：
哦，原来你也叫池啊！真巧！请问有什么问题或者话题想和我聊吗？我可以告诉你关于天气、新闻、历史等各种信息哦。
第二轮对话：
抱歉，我无法知道你的名字。你可以告诉我你的名字，这样我就能记住了！有什么其他问题或者话题想和我聊吗？


#### 对话字符缓存存储
使用对话字符缓存记忆，内存将限制保存的token数量。如果字符数量超出指定数目，它会切掉这个对话的早期部分 以保留与最近的交流相对应的字符数量，但不超过字符限制。

In [38]:
from langchain.llms import OpenAI
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})

{'history': 'AI: 轻舟已过万重山。'}

### 模型链 Chains
